In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

## Support Vector Machine (支持向量机)
过去的分类我们都是用线性函数来进行分类，如下图所示
然而，现实的场景往往难以利用一刀切的方式对目标进行分类，如下图所示
SVM便是为此而生，若我们将数据的维度提升，如下图所示
若原始数据为2维(2个特征)，我们需要提升到三维，便能以一刀切的方式对目标做分类，我们称将维度提升为kernel，维度提升后能一刀切的平面称为hyperplane，我们希望数据距离这个hyperplane越远越好，这个距离称为margin  
首先，我们假设SVM的模型为$f(w^Tx+b)$，若$w^Tx+b<0$则label为-1，反之$w^Tx+b>0$ label则为1，我们定义label为0或1而不是-1或1是为了之后方便计算，我们定义margin为$label*(w^Tx+b)$，这样的话，不管类别为何，当距离越远$label*(w^Tx+b)$会是正数且越来越大，任意点到平面$w^T+b$的距离为$\frac{label*(w^Tx+b)}{\left \| w \right \|}$，SVM求超平面的优化问题可以写成$max(\frac{min(label*(w^Tx+b))}{\left \| w \right \|})$，取min代表SVM只对离超平面最近的点做计算，正规一点可以写成
$$
arg \:  \min_{w,b} \frac{1}{2} || w ||^2  \\
subject\:to\:label(w^Tx+b)\geq c
$$
$\frac{1}{2}$为了之后推倒方便，$c$为距离超参数，接下来要求对偶形式，需要先知道拉格朗日乘子法
$$
L(w,b,a)= \frac{1}{2} || w ||^2  - \sum_{n}^{N}a_n*(label(w^Tx_n+b)- c)
$$
其中$a_n>0$代表乘子，我们在最小化L的情况下(题目要求)，乘子要最大化，如果约束条件成立，L为非负的，代表是可以求最小值的。  
为了得到小化L，需对w和b求导，分别为
$$
w=\sum_{n}^{N}a_nlabel_nx_n \\
0=\sum_{n}^{N}a_nlabel_n
$$
进而
$$
w^Tx+b=\sum_{n}^{N}a_nlabel_nx_n^Tx+b
$$
代入L(w,b,a)后，得到对偶问题
$$
max\: L(a)=\sum_{n}^{N}a_n-\frac{1}{2}\sum_{n}^{N}\sum_{m}^{N}a_na_mlabel_nlabel_mx_n^Tx_m \\
subject\: to\: a_n\geq 0, \sum_{n}^{N}a_nlabel_n=0
$$
或是
$$
min\: L(a)=\frac{1}{2}\sum_{n}^{N}\sum_{m}^{N}a_na_mlabel_nlabel_mx_n^Tx_m - \sum_{n}^{N}a_n \\
subject\: to\: a_n\geq 0, \sum_{n}^{N}a_nlabel_n=0
$$
只有$a$这个乘子，我们比较容易求极值，但是仍然需要大量计算，我们使用SMO算法求极值
### 优缺点

优点：精准度高、计算量不高、对结果可解释性高  
缺点：对超参数以及kernel设置敏感、适合处理二分类问题、难以训练大规模样本

接下来，我们从最简单的线性SVM分类器开始

In [2]:
def loadDataSet(fileName):
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

def selectJrand(i,m):
    j=i #we want to select any J not equal to i
    while (j==i):
        j = int(random.uniform(0,m))
    return j

def clipAlpha(aj,H,L):
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj

以上三个函数为helper函数，loadDataSet载入数据以及label，selectJrand产生从0~m不等于i的随机数，clipAlpha回传L<aj<H的值，接下来，是SMO算法的核心代码

In [4]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = np.mat(dataMatIn); labelMat = np.mat(classLabels).transpose()
    b = 0; m,n = np.shape(dataMatrix)
    alphas = np.mat(zeros((m,1)))
    iter = 0
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m):
            fXi = float(np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
            Ei = fXi - float(labelMat[i])#if checks if an example violates KKT conditions
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i,m)
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: print "L==H"; continue
                eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T
                if eta >= 0: print "eta>=0"; continue
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                alphas[j] = clipAlpha(alphas[j],H,L)
                if (abs(alphas[j] - alphaJold) < 0.00001): print "j not moving enough"; continue
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])#update i by the same amount as j
                                                                        #the update is in the oppostie direction
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]): b = b1
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2)/2.0
                alphaPairsChanged += 1
                print "iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
        if (alphaPairsChanged == 0): iter += 1
        else: iter = 0
        print "iteration number: %d" % iter
    return b,alphas

([[3.542485, 1.977398],
  [3.018896, 2.556416],
  [7.55151, -1.58003],
  [2.114999, -0.004466],
  [8.127113, 1.274372],
  [7.108772, -0.986906],
  [8.610639, 2.046708],
  [2.326297, 0.265213],
  [3.634009, 1.730537],
  [0.341367, -0.894998],
  [3.125951, 0.293251],
  [2.123252, -0.783563],
  [0.887835, -2.797792],
  [7.139979, -2.329896],
  [1.696414, -1.212496],
  [8.117032, 0.623493],
  [8.497162, -0.266649],
  [4.658191, 3.507396],
  [8.197181, 1.545132],
  [1.208047, 0.2131],
  [1.928486, -0.32187],
  [2.175808, -0.014527],
  [7.886608, 0.461755],
  [3.223038, -0.552392],
  [3.628502, 2.190585],
  [7.40786, -0.121961],
  [7.286357, 0.251077],
  [2.301095, -0.533988],
  [-0.232542, -0.54769],
  [3.457096, -0.082216],
  [3.023938, -0.057392],
  [8.015003, 0.885325],
  [8.991748, 0.923154],
  [7.916831, -1.781735],
  [7.616862, -0.217958],
  [2.450939, 0.744967],
  [7.270337, -2.507834],
  [1.749721, -0.961902],
  [1.803111, -0.176349],
  [8.804461, 3.044301],
  [1.231257, -0.568573],